# Sparkify Project Workspace
This workspace contains a tiny subset (128MB) of the full dataset available (12GB). Feel free to use this workspace to build your project, or to explore a smaller subset with Spark before deploying your cluster on the cloud. Instructions for setting up your Spark cluster is included in the last lesson of the Extracurricular Spark Course content.

You can follow the steps below to guide your data analysis and model building portion of this project.

# Bussiness Understanding

The dataset seems to belong to some digial music streaming Platform. Whenever a user interacts with the music streaming service the logs get generated like timestamp and the ahe activity page(i.e logOut, like, logIn etc). Tthere are basically types of user 1. free -tier and 2. subscription based. so based on the activity of the user we need to figure out which user is likely to churn out. So that, some kind of discounts can be rolled out to that specific users.

# Data Understanding

the datat is contained in a file `mini_sparkify_event_data.json` 
Data description :
* artist: string name of artist
* auth : String authencation levels
* firstName: string first name of user
* gender: string Gender of user
* itemInSession: long log count in the session
* lastName: string last name of the user
* length: double length of the song
* level: string type of subscription
* location: string location of user
* method: string tupe of request
* page: string type of interaction
* registration: long registration timestamp
* sessionId: long session id of the user
* song: string name of song
* status: long http status code
* ts: long timestamp
* userAgent: string agent used by user for straeming service
* userId: string inique identifier of  the user


In [5]:
# import libraries
from pyspark.sql import SparkSession,Window
from pyspark.sql.functions import avg, col, concat, desc, explode, lit, min, max, split, udf, count, when, isnull, collect_list
from pyspark.sql.types import *
from pyspark.ml.feature import VectorAssembler, StandardScaler
from pyspark.storagelevel import StorageLevel
from pyspark.sql.functions import *
from pyspark.ml.classification import LogisticRegression, GBTClassifier
from pyspark.ml.evaluation import BinaryClassificationEvaluator
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder
from pyspark.sql import functions as f
import seaborn as sns
import matplotlib.pyplot as plt

%matplotlib inline

In [6]:
import pandas as pd
%matplotlib inline
pd.set_option('display.max_columns', None)  
pd.set_option('display.expand_frame_repr', False)
pd.set_option('max_colwidth', -1)

C:\Users\PRAVEEN KUMAR\Anaconda3\envs\pyspark_env\lib\site-packages\ipykernel_launcher.py:5: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.
  """


In [7]:
# create a Spark session
sc =  SparkSession \
    .builder \
    .appName("Sparkify") \
    .getOrCreate()

# Load and Clean Dataset
In this workspace, the mini-dataset file is `mini_sparkify_event_data.json`. Load and clean the dataset, checking for invalid or missing data - for example, records without userids or sessionids. 

In [47]:
filename = 'mini_sparkify_event_data.json'
df = sc.read.json(filename)
df.printSchema()

root
 |-- artist: string (nullable = true)
 |-- auth: string (nullable = true)
 |-- firstName: string (nullable = true)
 |-- gender: string (nullable = true)
 |-- itemInSession: long (nullable = true)
 |-- lastName: string (nullable = true)
 |-- length: double (nullable = true)
 |-- level: string (nullable = true)
 |-- location: string (nullable = true)
 |-- method: string (nullable = true)
 |-- page: string (nullable = true)
 |-- registration: long (nullable = true)
 |-- sessionId: long (nullable = true)
 |-- song: string (nullable = true)
 |-- status: long (nullable = true)
 |-- ts: long (nullable = true)
 |-- userAgent: string (nullable = true)
 |-- userId: string (nullable = true)



In [27]:
def show_df(df):
    """
        ARGS:
            - RDD
        RETURNS:
            - Prints dataframe
    """
    return pd.DataFrame(df.take(5),columns=df.columns).head()
show_df(df)

,artist,auth,firstName,gender,itemInSession,lastName,length,level,location,method,page,registration,sessionId,song,status,ts,userAgent,userId
0,Martha Tilston,Logged In,Colin,M,50,Freeman,277.89016,paid,"Bakersfield, CA",PUT,NextSong,1538173362000,29,Rockpools,200,1538352117000,Mozilla/5.0 (Windows NT 6.1; WOW64; rv:31.0) Gecko/20100101 Firefox/31.0,30
1,Five Iron Frenzy,Logged In,Micah,M,79,Long,236.09424,free,"Boston-Cambridge-Newton, MA-NH",PUT,NextSong,1538331630000,8,Canada,200,1538352180000,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/37.0.2062.103 Safari/537.36""",9
2,Adam Lambert,Logged In,Colin,M,51,Freeman,282.82730,paid,"Bakersfield, CA",PUT,NextSong,1538173362000,29,Time For Miracles,200,1538352394000,Mozilla/5.0 (Windows NT 6.1; WOW64; rv:31.0) Gecko/20100101 Firefox/31.0,30
3,Enigma,Logged In,Micah,M,80,Long,262.71302,free,"Boston-Cambridge-Newton, MA-NH",PUT,NextSong,1538331630000,8,Knocking On Forbidden Doors,200,1538352416000,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/37.0.2062.103 Safari/537.36""",9
4,Daft Punk,Logged In,Colin,M,52,Freeman,223.60771,paid,"Bakersfield, CA",PUT,NextSong,1538173362000,29,Harder Better Faster Stronger,200,1538352676000,Mozilla/5.0 (Windows NT 6.1; WOW64; rv:31.0) Gecko/20100101 Firefox/31.0,30


In [28]:
def df_dimentions(df):
    """
        Prints number of Rows and Columns
        ARGS:
            - Dataframe
    """
    print(f"row count: {df.count()}, column count: {len(df.columns)}")
df_dimentions(df)

row count: 286500, column count: 18


In [48]:
## remove special characters from column
def remove_special_chars(df):
    """
        removes special characters from data
        ARGS:
            - Dataframe
        RETURNS:
            - clear Dataframe
    """
    for field in df.schema.fields:
        if field.dataType == StringType():
            df = df.withColumn(field.name,regexp_replace(field.name,'[^a-zA-Z0-9\,\-]',''))
    
    return df
df = remove_special_chars(df)
show_df(df)

,artist,auth,firstName,gender,itemInSession,lastName,length,level,location,method,page,registration,sessionId,song,status,ts,userAgent,userId
0,MarthaTilston,LoggedIn,Colin,M,50,Freeman,277.89016,paid,"Bakersfield,CA",PUT,NextSong,1538173362000,29,Rockpools,200,1538352117000,Mozilla50WindowsNT61WOW64rv310Gecko20100101Firefox310,30
1,FiveIronFrenzy,LoggedIn,Micah,M,79,Long,236.09424,free,"Boston-Cambridge-Newton,MA-NH",PUT,NextSong,1538331630000,8,Canada,200,1538352180000,"Mozilla50WindowsNT61WOW64AppleWebKit53736KHTML,likeGeckoChrome3702062103Safari53736",9
2,AdamLambert,LoggedIn,Colin,M,51,Freeman,282.82730,paid,"Bakersfield,CA",PUT,NextSong,1538173362000,29,TimeForMiracles,200,1538352394000,Mozilla50WindowsNT61WOW64rv310Gecko20100101Firefox310,30
3,Enigma,LoggedIn,Micah,M,80,Long,262.71302,free,"Boston-Cambridge-Newton,MA-NH",PUT,NextSong,1538331630000,8,KnockingOnForbiddenDoors,200,1538352416000,"Mozilla50WindowsNT61WOW64AppleWebKit53736KHTML,likeGeckoChrome3702062103Safari53736",9
4,DaftPunk,LoggedIn,Colin,M,52,Freeman,223.60771,paid,"Bakersfield,CA",PUT,NextSong,1538173362000,29,HarderBetterFasterStronger,200,1538352676000,Mozilla50WindowsNT61WOW64rv310Gecko20100101Firefox310,30


In [49]:
def convert_unixtimestamp(df,ts_col):
    """
        converts unix timestamp to date time
        ARGS:
            - df: Dataframe with Unixtimestamp
            - ts: unixtimestamp column name
        RETURNS:
            - dataframe with new datetime column
    """
    df = df.withColumn("dateTime",from_unixtime(col(ts_col).cast(LongType())/1000).cast(TimestampType()))
    df = df.withColumn("month",month(col('dateTime')))
    df = df.withColumn("date",from_unixtime(col(ts_col)/1000).cast(DateType()))
    df = df.withColumn("location",split(col('location'),',').getItem(1))
    df = df.filter(col('userId').isNotNull())
    return df


In [81]:
def convert_to_pandas_df(df):
    """
        converts spark to pandas dataframe
        ARGS:
            - Spark dataframe
        RETURNS:
            - Pandas Datframe
    """
    return pd.DataFrame(data=df,columns=df.columns)

In [82]:
df = convert_unixtimestamp(df,'ts')

In [ ]:
convert_to_pandas_df(df.take(100))

In [69]:
df.filter(col('userId').isNull()).count()

0

In [ ]:
# Dropping the missing User's ID rows
df = df.filter(df.userId != '')
# Check
df.filter(df.userId == '').count()

# Exploratory Data Analysis
When you're working with the full dataset, perform EDA by loading a small subset of the data and doing basic manipulations within Spark. In this workspace, you are already provided a small subset of data you can explore.

### Define Churn

Once you've done some preliminary analysis, create a column `Churn` to use as the label for your model. I suggest using the `Cancellation Confirmation` events to define your churn, which happen for both paid and free users. As a bonus task, you can also look into the `Downgrade` events.

### Explore Data
Once you've defined churn, perform some exploratory data analysis to observe the behavior for users who stayed vs users who churned. You can start by exploring aggregates on these two groups of users, observing how much of a specific action they experienced per a certain time unit or number of songs played.

In [52]:
def distinct_attributes(df,col_name):
    """
        Prints all the distinct attributes/values of a single column
        ARGS:
            - df: Dataframe
            - col_name: Name of the column for distinct attributes
        
    """
    print(df.select(col_name).distinct().collect())

In [53]:
def categorical_columns(df):
    """
        prints distinct values of categorical columns
        ARGS:
            - df: Dataframe
    """
    for field in df.schema.fields:
        if ((field.dataType == StringType()) and (field.name not in ['userId','lastName','userAgent','firstName','artist','song'] )):
            print(field.name)
            distinct_attributes(df,field.name)
            print("*"*50)
categorical_columns(df)

auth
[Row(auth='Cancelled'), Row(auth='Guest'), Row(auth='LoggedIn'), Row(auth='LoggedOut')]
**************************************************
gender
[Row(gender='F'), Row(gender=None), Row(gender='M')]
**************************************************
level
[Row(level='free'), Row(level='paid')]
**************************************************
location
[Row(location='UT-ID'), Row(location='AZ'), Row(location='SC'), Row(location='GA-AL'), Row(location='LA'), Row(location='RI-MA'), Row(location='NJ'), Row(location='IL-MO'), Row(location='MN-WI'), Row(location='OR'), Row(location='SC-NC'), Row(location='PA-NJ'), Row(location='MD-WV'), Row(location=None), Row(location='VA'), Row(location='NY-NJ-PA'), Row(location='KY'), Row(location='NH'), Row(location='MI'), Row(location='NV'), Row(location='WI'), Row(location='CA'), Row(location='CT'), Row(location='MT'), Row(location='NC'), Row(location='MD'), Row(location='NE-IA'), Row(location='IL'), Row(location='WA'), Row(location='MS'), Row(lo

In [ ]:
df.filter(df.page=="CancellationConfirmation").select("userId").dropDuplicates().show(10)

In [57]:
def create_label(df):
    """
        Create label columns containing 1,0 for cancellation page and auth is cancled
        ARGS:
            - df: Dataframe 
        RETURNS:
            - Dataframe with labeled columns
    """
    
    flag_cancelation_event = udf(lambda x: 1 if x == "CancellationConfirmation" else 0, IntegerType())
    # apply to the dataframe
    df = df.withColumn("label", flag_cancelation_event("page"))
    #Define window bounds
    windowval = Window.partitionBy("userId").rangeBetween(Window.unboundedPreceding, Window.unboundedFollowing)
    # Applying the window
    df = df.withColumn("label", sum("label").over(windowval))
    return df
df = create_label(df)
show_df(df)

,userId,artist,auth,firstName,gender,itemInSession,lastName,length,level,location,method,page,registration,sessionId,song,status,ts,userAgent,dateTime,month,date,label
0,100010,SleepingWithSirens,LoggedIn,Darianna,F,0,Carpenter,202.97098,free,CT,PUT,NextSong,1538016340000,31,CaptainTyinKnotsVSMrWalkwayNoWay,200,1539003534000,"Mozilla50iPhoneCPUiPhoneOS712likeMacOSXAppleWebKit537512KHTML,likeGeckoVersion70Mobile11D257Safari953753",2018-10-08 18:28:54,10,2018-10-08,0
1,100010,FrancescaBattistelli,LoggedIn,Darianna,F,1,Carpenter,196.54485,free,CT,PUT,NextSong,1538016340000,31,BeautifulBeautifulAlbum,200,1539003736000,"Mozilla50iPhoneCPUiPhoneOS712likeMacOSXAppleWebKit537512KHTML,likeGeckoVersion70Mobile11D257Safari953753",2018-10-08 18:32:16,10,2018-10-08,0
2,100010,Brutha,LoggedIn,Darianna,F,2,Carpenter,263.13098,free,CT,PUT,NextSong,1538016340000,31,ShesGone,200,1539003932000,"Mozilla50iPhoneCPUiPhoneOS712likeMacOSXAppleWebKit537512KHTML,likeGeckoVersion70Mobile11D257Safari953753",2018-10-08 18:35:32,10,2018-10-08,0
3,100010,None,LoggedIn,Darianna,F,3,Carpenter,NaN,free,CT,PUT,ThumbsUp,1538016340000,31,None,307,1539003933000,"Mozilla50iPhoneCPUiPhoneOS712likeMacOSXAppleWebKit537512KHTML,likeGeckoVersion70Mobile11D257Safari953753",2018-10-08 18:35:33,10,2018-10-08,0
4,100010,JoshRitter,LoggedIn,Darianna,F,4,Carpenter,316.23791,free,CT,PUT,NextSong,1538016340000,31,FolkBloodbath,200,1539004195000,"Mozilla50iPhoneCPUiPhoneOS712likeMacOSXAppleWebKit537512KHTML,likeGeckoVersion70Mobile11D257Safari953753",2018-10-08 18:39:55,10,2018-10-08,0


In [73]:
# create dataframe for plotting count of different labels of all the users
label_df = convert_to_pandas_df(df.dropDuplicates(['userId']).groupby(['label']).count().show())
label_df.head()

+-----+-----+
|label|count|
+-----+-----+
|    0|  173|
|    1|   52|
+-----+-----+



""


In [68]:
# Number of users in Dataset
df.select('userId').dropDuplicates().count()

225

In [67]:
df = df.dropna()

In [ ]:
# types of user churn by gender
gender_df = df.groupby(['label', 'gender']).count().take(10)
gender_df = convert_to_pandas_df(gender_df)
gender_df.head()

In [76]:
gender_df.head()

""


In [ ]:
df.where("artist is null").show(20)

In [ ]:
set(df.select('page').distinct().collect())-set(df.where("artist is null").select('page').distinct().collect())

In [ ]:
distinct_attributes(df,"auth")    

In [ ]:
distinct_attributes(df,'page')

In [ ]:
def count_missings(spark_df,sort=True):
    """
        Counts number of nulls and nans in each column
        ARGS:
            - spark_df: RDD dataframe 
    """
    df = spark_df.select([f.count(f.when(f.isnan(c) | f.isnull(c), c)).alias(c) for (c,c_type) in spark_df.dtypes if c_type not in ('timestamp', 'string', 'date')]).toPandas()

    if len(df) == 0:
        print("There are no any missing values!")
        return None

    if sort:
        return df.rename(index={0: 'missing_value_count'}).T.sort_values("missing_value_count",ascending=False)

    return df

In [ ]:
count_missings(df)

In [ ]:
df.select('userId').dropDuplicates().distinct().count()

In [ ]:
def plot_spark_df(df,col_name):
    """
        Plots histogram for from spark dataframe
        ARGS:
            - df: Spark dataframe
    """
    pd_df = df.toPandas()
    plt.figure(figsize=(15,5))
    sns.barplot(x=col_name,data=pd_df)

In [ ]:
pd_df = df.toPandas()
sns.barplot(x='auth',data=pd_df)

# Feature Engineering
Once you've familiarized yourself with the data, build out the features you find promising to train your model on. To work with the full dataset, you can follow the following steps.
- Write a script to extract the necessary features from the smaller subset of data
- Ensure that your script is scalable, using the best practices discussed in Lesson 3
- Try your script on the full data set, debugging your script if necessary

If you are working in the classroom workspace, you can just extract features based on the small subset of data contained here. Be sure to transfer over this work to the larger dataset when you work on your Spark cluster.

In [ ]:
df = convert_unixtimestamp(df,'ts')
df = calculate_active_duration(df)
show_df(df)

In [ ]:
def calculate_active_duration(df):
    """
        Calculates number of days a user is active since the day he registerd
        ARGS:
            - dataframe
        REURNS:
            - Dataframe with new colum activeDuration
    """
    df = df.withColumn('regDatetime',from_unixtime(col('registration')/1000).cast(DateType()))
    user_duration_df = df.groupBy('UserId','regDatetime').agg(max('date').alias('lastInterationDate')) 
    diff  = (f.unix_timestamp('lastInterationDate')-f.unix_timestamp('regDatetime'))
    duration_df = user_duration_df.withColumn('activeDuration',diff/86400)
    
    df = df.join(duration_df,on='UserId')
    return df  

In [ ]:
def user_session_sum():
    """
        Create a column containing the sum of ItemInSession column
        RETURNS:
            - Dataframe containing column sumItemInSession 
    """
    item_sum_df = df.groupby('userId').agg(sum(col('itemInSession')).alias('sumItemInSession'))
    df =  df.join(item_sum_df,on='userId')
    return df

# Modeling
Split the full dataset into train, test, and validation sets. Test out several of the machine learning methods you learned. Evaluate the accuracy of the various models, tuning parameters as necessary. Determine your winning model based on test accuracy and report results on the validation set. Since the churned users are a fairly small subset, I suggest using F1 score as the metric to optimize.

# Final Steps
Clean up your code, adding comments and renaming variables to make the code easier to read and maintain. Refer to the Spark Project Overview page and Data Scientist Capstone Project Rubric to make sure you are including all components of the capstone project and meet all expectations. Remember, this includes thorough documentation in a README file in a Github repository, as well as a web app or blog post.